# リソース構築

●●●リソースの情報の保存用変数●●●

In [6]:
res_info = {'vender': 'aws'}


## EKSクラスタの構築(kubernetesクラスタ)

In [4]:
clusterName = 'aws-eks-200214'
nodeGroupName = 'ng-200214'

In [7]:
res_info['clusterName']=clusterName
res_info['nodeGroupName']=nodeGroupName

In [4]:
!eksctl create cluster \
    --vpc-cidr 10.0.0.0/16 \
    --vpc-nat-mode HighlyAvailable \
    --name {clusterName} \
    --version 1.14 \
    --nodegroup-name {nodeGroupName} \
    --node-type t3.small \
    --nodes 2 \
    --zones=us-west-2b,us-west-2c \
    --managed

[ℹ]  eksctl version 0.13.0
[ℹ]  using region us-west-2
[ℹ]  subnets for us-west-2b - public:10.0.0.0/19 private:10.0.64.0/19
[ℹ]  subnets for us-west-2c - public:10.0.32.0/19 private:10.0.96.0/19
[ℹ]  using Kubernetes version 1.14
[ℹ]  creating EKS cluster "aws-eks-200130" in "us-west-2" region with managed nodes
[ℹ]  will create 2 separate CloudFormation stacks for cluster itself and the initial managed nodegroup
[ℹ]  if you encounter any issues, check CloudFormation console or try 'eksctl utils describe-stacks --region=us-west-2 --cluster=aws-eks-200130'
[ℹ]  CloudWatch logging will not be enabled for cluster "aws-eks-200130" in "us-west-2"
[ℹ]  you can enable it with 'eksctl utils update-cluster-logging --region=us-west-2 --cluster=aws-eks-200130'
[ℹ]  Kubernetes API endpoint access will use default of {publicAccess=true, privateAccess=false} for cluster "aws-eks-200130" in "us-west-2"
[ℹ]  2 sequential tasks: { create cluster control plane "aws-eks-200130", create managed nodegroup

クラスターができていることを確認する

In [5]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   172.20.0.1   <none>        443/TCP   18m


## EFSの構築(NFS)

■IAMロールの一覧取得

■NodeInstanceRoleのロール名をコピー（eksctl-aws-eks-0108-nodegroup-ng-NodeInstanceRole-XGXVUO8O861C）

In [11]:
row = !aws iam list-roles | grep '{clusterName}-nodegroup-n-NodeInstanceRole-'


import re

m = re.search(r'"RoleName": "(.*)",', row[0])
NodeInstanceRole = m.group(1)
print(NodeInstanceRole)

res_info['NodeInstanceRole']=NodeInstanceRole

eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5


■ロールにEFSのポリシーをアタッチ

In [29]:
!aws iam attach-role-policy --role-name \
{NodeInstanceRole} \
--policy-arn arn:aws:iam::aws:policy/AmazonElasticFileSystemFullAccess

■!awsを入れて、再実行→成功 (プレビュー版ではなく、正式版になっている)

In [30]:
!aws efs create-file-system \
--creation-token "EFS-topse" \
--performance-mode "generalPurpose"

{
    "OwnerId": "565991835156",
    "CreationToken": "EFS-topse",
    "FileSystemId": "fs-cc76c766",
    "CreationTime": 1580382776.0,
    "LifeCycleState": "creating",
    "NumberOfMountTargets": 0,
    "SizeInBytes": {
        "Value": 0,
        "ValueInIA": 0,
        "ValueInStandard": 0
    },
    "PerformanceMode": "generalPurpose",
    "Encrypted": false,
    "ThroughputMode": "bursting",
    "Tags": []
}


■EFSの作成確認とFileSystemIdの取り出し

In [12]:
row = !aws efs describe-file-systems --creation-token "EFS-topse" | grep FileSystemId
print(row)

import re
m = re.search(r'"FileSystemId": "(.*)",', row[0])
FileSystemId = m.group(1)
print(FileSystemId)

#リソース情報に追加
res_info['FileSystemId'] = FileSystemId

print(res_info)

['            "FileSystemId": "fs-83a42529",']
fs-83a42529
{'vender': 'aws', 'clusterName': 'aws-eks-200214', 'nodeGroupName': 'ng-200214', 'workerRolearn': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5', 'NodeInstanceRole': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5', 'FileSystemId': 'fs-83a42529'}


■Nameタグ付け

In [32]:
!aws efs create-tags --file-system-id {FileSystemId} --tags "Key=Name,Value={clusterName}"

■Nameタグ付け確認

In [33]:
!aws efs describe-tags --file-system-id {FileSystemId}

{
    "Tags": [
        {
            "Key": "Name",
            "Value": "aws-eks-200130"
        }
    ]
}


■eksのセキュリティグループとサブネットIDの確認

クラスターセキュリティーグループのIDの取得

In [34]:
row = !aws eks describe-cluster --name {clusterName}

import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"

clusterSecurityGroupId =json.loads(''.join(row))['cluster']['resourcesVpcConfig']['clusterSecurityGroupId']
pprint.pprint(clusterSecurityGroupId)


'sg-0dc32db2a557106b7'


■マウントターゲット作成

プライベートのサブネットを指定する

In [35]:
!aws efs create-mount-target \
--file-system-id {FileSystemId} \
--subnet-id {privateSubnets['Subnets'][0]['SubnetId']} \
--security-group {clusterSecurityGroupId}

{
    "OwnerId": "565991835156",
    "MountTargetId": "fsmt-413e7de9",
    "FileSystemId": "fs-cc76c766",
    "SubnetId": "subnet-0880877de3ef6143a",
    "LifeCycleState": "creating",
    "IpAddress": "10.0.97.63",
    "NetworkInterfaceId": "eni-0bb220dc84b6a7897"
}


In [36]:
!aws efs create-mount-target \
--file-system-id {FileSystemId} \
--subnet-id {privateSubnets['Subnets'][1]['SubnetId']} \
--security-group {clusterSecurityGroupId}

{
    "OwnerId": "565991835156",
    "MountTargetId": "fsmt-423e7dea",
    "FileSystemId": "fs-cc76c766",
    "SubnetId": "subnet-0606ee1fa3fc64dff",
    "LifeCycleState": "creating",
    "IpAddress": "10.0.89.105",
    "NetworkInterfaceId": "eni-0454599c821b9636d"
}


■マウントターゲット作成確認

In [37]:
!aws efs describe-mount-targets \
--file-system-id {FileSystemId}

{
    "MountTargets": [
        {
            "OwnerId": "565991835156",
            "MountTargetId": "fsmt-413e7de9",
            "FileSystemId": "fs-cc76c766",
            "SubnetId": "subnet-0880877de3ef6143a",
            "LifeCycleState": "creating",
            "IpAddress": "10.0.97.63",
            "NetworkInterfaceId": "eni-0bb220dc84b6a7897"
        },
        {
            "OwnerId": "565991835156",
            "MountTargetId": "fsmt-423e7dea",
            "FileSystemId": "fs-cc76c766",
            "SubnetId": "subnet-0606ee1fa3fc64dff",
            "LifeCycleState": "creating",
            "IpAddress": "10.0.89.105",
            "NetworkInterfaceId": "eni-0454599c821b9636d"
        }
    ]
}


セキュリティグループの取り出し

In [38]:
row = !aws ec2 describe-security-groups
import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
SecurityGrrowoups =json.loads(''.join(row))

ControlPlaneSecurityGroup = ""
ClusterSharedNodeSecurityGroup = ""

for sg in SecurityGrrowoups['SecurityGroups']:
    
    if  "ControlPlaneSecurityGroup" in sg["GroupName"]:
        ControlPlaneSecurityGroup = sg["GroupId"]
    
    if "ClusterSharedNodeSecurityGroup" in sg["GroupName"]:
        ClusterSharedNodeSecurityGroup = sg["GroupId"]

print(ControlPlaneSecurityGroup)
print(ClusterSharedNodeSecurityGroup)

sg-04ed0c1531875e1dd
sg-074190909e2319be3


セキュリティグループ間のNFSインバウンドルールの追加

In [39]:
!aws ec2 authorize-security-group-ingress --group-id {ControlPlaneSecurityGroup} \
--protocol tcp --port 2049 --source-group {ClusterSharedNodeSecurityGroup}

In [40]:
!aws ec2 authorize-security-group-ingress --group-id {ClusterSharedNodeSecurityGroup} \
--protocol tcp --port 2049 --source-group {ControlPlaneSecurityGroup}

efs-provisioner を使用して、Kubernetes の永続ボリュームとして EFS ストレージをマウントすることにより、Amazon EFS を Amazon EKS ポッドで使用できるようにする。
https://aws.amazon.com/jp/premiumsupport/knowledge-center/eks-pods-efs/

In [13]:
region="us-west-2"

#リソース情報に追加
res_info['region'] = "us-west-2"

print(res_info)

{'vender': 'aws', 'clusterName': 'aws-eks-200214', 'nodeGroupName': 'ng-200214', 'workerRolearn': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5', 'NodeInstanceRole': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5', 'FileSystemId': 'fs-83a42529', 'region': 'us-west-2'}


## RDS構築(DB)

プライベートなサブネットをサブネットグループとしてまとめる

In [49]:
SubnetGroupName = "mysqlsubnetgroup"
print(SubnetGroupName)

mysqlsubnetgroup


In [57]:
!aws rds create-db-subnet-group \
        --db-subnet-group-name "{SubnetGroupName}" \
        --db-subnet-group-description "mysql_db_subnet_group" \
        --subnet-ids "{privateSubnets['Subnets'][0]['SubnetId']}" "{privateSubnets['Subnets'][1]['SubnetId']}"

{
    "DBSubnetGroup": {
        "DBSubnetGroupName": "mysqlsubnetgroup",
        "DBSubnetGroupDescription": "mysql_db_subnet_group",
        "VpcId": "vpc-031af53d13bacea11",
        "SubnetGroupStatus": "Complete",
        "Subnets": [
            {
                "SubnetIdentifier": "subnet-0606ee1fa3fc64dff",
                "SubnetAvailabilityZone": {
                    "Name": "us-west-2b"
                },
                "SubnetStatus": "Active"
            },
            {
                "SubnetIdentifier": "subnet-0880877de3ef6143a",
                "SubnetAvailabilityZone": {
                    "Name": "us-west-2c"
                },
                "SubnetStatus": "Active"
            }
        ],
        "DBSubnetGroupArn": "arn:aws:rds:us-west-2:565991835156:subgrp:mysqlsubnetgroup"
    }
}


RDSの名前などを定義し、RDSを作成

In [2]:
db_name = "RDS-topse"
db_user = "root"
db_password = "password"

In [58]:
!aws rds create-db-instance \
    --db-instance-identifier "{db_name}" \
    --allocated-storage 5 \
    --db-instance-class db.t2.micro \
    --engine mysql \
    --master-username "{db_user}" \
    --master-user-password "{db_password}" \
    --db-subnet-group-name  "{SubnetGroupName}" \
    --vpc-security-group-ids "{ClusterSharedNodeSecurityGroup}" \
    --multi-az


{
    "DBInstance": {
        "DBInstanceIdentifier": "rds-topse",
        "DBInstanceClass": "db.t2.micro",
        "Engine": "mysql",
        "DBInstanceStatus": "creating",
        "MasterUsername": "root",
        "AllocatedStorage": 5,
        "PreferredBackupWindow": "06:09-06:39",
        "BackupRetentionPeriod": 1,
        "DBSecurityGroups": [],
        "VpcSecurityGroups": [
            {
                "VpcSecurityGroupId": "sg-074190909e2319be3",
                "Status": "active"
            }
        ],
        "DBParameterGroups": [
            {
                "DBParameterGroupName": "default.mysql5.7",
                "ParameterApplyStatus": "in-sync"
            }
        ],
        "DBSubnetGroup": {
            "DBSubnetGroupName": "mysqlsubnetgroup",
            "DBSubnetGroupDescription": "mysql_db_subnet_group",
            "VpcId": "vpc-031af53d13bacea11",
            "SubnetGroupStatus": "Complete",
            "Subnets": [
                {
                 

DBの作成が終わるのをまつ


In [3]:
!aws rds wait db-instance-available --db-instance-identifier "{db_name}"

後で使うのでエンドポイントを取得しておく

In [14]:
row = !aws rds describe-db-instances \
    --db-instance-identifier "{db_name}"
import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
DBInstances =json.loads(''.join(row))

db_address = DBInstances["DBInstances"][0]["Endpoint"]["Address"]

print(db_address)

rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com


In [15]:
res_info['db_address']=db_address
print(res_info)

{'vender': 'aws', 'clusterName': 'aws-eks-200214', 'nodeGroupName': 'ng-200214', 'workerRolearn': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5', 'NodeInstanceRole': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5', 'FileSystemId': 'fs-83a42529', 'region': 'us-west-2', 'db_address': 'rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com'}


## ALBの構築

IngressからALBを作成するには、IAMロールやら、AWS ALB Ingress Controllerやらの設定が必要。


https://docs.aws.amazon.com/ja_jp/eks/latest/userguide/alb-ingress.html

ロードバランサーに使用する VPC のサブネットにタグを付け、そのサブネットを使用できることを ALB Ingress Controller に伝えます。詳細については、「サブネットのタグ付け要件」を参照してください。

PublicとPrivateを別々に取得

In [19]:
row = !aws ec2 describe-subnets --filters "Name=tag:Name,Values=*{clusterName}*Public*"

import json
import pprint

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
publicSubnets = json.loads(''.join(row))
pprint.pprint(publicSubnets)


{'Subnets': [{'AssignIpv6AddressOnCreation': False,
              'AvailabilityZone': 'us-west-2b',
              'AvailabilityZoneId': 'usw2-az2',
              'AvailableIpAddressCount': 8178,
              'CidrBlock': '10.0.0.0/19',
              'DefaultForAz': False,
              'Ipv6CidrBlockAssociationSet': [],
              'MapPublicIpOnLaunch': False,
              'OwnerId': '565991835156',
              'State': 'available',
              'SubnetArn': 'arn:aws:ec2:us-west-2:565991835156:subnet/subnet-00edd2cff6e184d73',
              'SubnetId': 'subnet-00edd2cff6e184d73',
              'Tags': [{'Key': 'aws:cloudformation:logical-id',
                        'Value': 'SubnetPublicUSWEST2B'},
                       {'Key': 'kubernetes.io/role/elb', 'Value': '1'},
                       {'Key': 'aws:cloudformation:stack-id',
                        'Value': 'arn:aws:cloudformation:us-west-2:565991835156:stack/eksctl-aws-eks-200214-cluster/52b98e90-4f03-11ea-a7ce-0688c4290

In [7]:
row = !aws ec2 describe-subnets --filters "Name=tag:Name,Values=*{clusterName}*Private*"

import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
privateSubnets = json.loads(''.join(row))
pprint.pprint(privateSubnets)

{'Subnets': [{'AssignIpv6AddressOnCreation': False,
              'AvailabilityZone': 'us-west-2c',
              'AvailabilityZoneId': 'usw2-az3',
              'AvailableIpAddressCount': 8186,
              'CidrBlock': '10.0.96.0/19',
              'DefaultForAz': False,
              'Ipv6CidrBlockAssociationSet': [],
              'MapPublicIpOnLaunch': False,
              'OwnerId': '565991835156',
              'State': 'available',
              'SubnetArn': 'arn:aws:ec2:us-west-2:565991835156:subnet/subnet-0880877de3ef6143a',
              'SubnetId': 'subnet-0880877de3ef6143a',
              'Tags': [{'Key': 'aws:cloudformation:stack-name',
                        'Value': 'eksctl-aws-eks-200130-cluster'},
                       {'Key': 'kubernetes.io/cluster/aws-eks-200130',
                        'Value': 'shared'},
                       {'Key': 'aws:cloudformation:logical-id',
                        'Value': 'SubnetPrivateUSWEST2C'},
                       {'Key': 'alp

Kubernetes 用のタグはすでに付与されていることを確認した。

ワーカーノードインスタンスプロファイルに対して、ALBIngressControllerIAMPolicy という IAM ポリシーを作成し、ALB Ingress Controller がユーザーに代わって AWS API を呼び出せるようにします。次の AWS CLI コマンドを使用して、AWS アカウントで IAM ポリシーを作成します。GitHub でポリシードキュメントを表示することもできます。

GitHub からポリシードキュメントをダウンロードします。

In [9]:
!mkdir temp

In [10]:
!wget -O temp/iam-policy.json https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/iam-policy.json

--2020-01-30 11:06:42--  https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/iam-policy.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3546 (3.5K) [text/plain]
Saving to: ‘temp/iam-policy.json’

temp/iam-policy.jso 100%[===================>]   3.46K  --.-KB/s    in 0s      

2020-01-30 11:06:42 (24.9 MB/s) - ‘temp/iam-policy.json’ saved [3546/3546]



In [11]:
!cat temp/iam-policy.json

{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "acm:DescribeCertificate",
        "acm:ListCertificates",
        "acm:GetCertificate"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:AuthorizeSecurityGroupIngress",
        "ec2:CreateSecurityGroup",
        "ec2:CreateTags",
        "ec2:DeleteTags",
        "ec2:DeleteSecurityGroup",
        "ec2:DescribeAccountAttributes",
        "ec2:DescribeAddresses",
        "ec2:DescribeInstances",
        "ec2:DescribeInstanceStatus",
        "ec2:DescribeInternetGateways",
        "ec2:DescribeNetworkInterfaces",
        "ec2:DescribeSecurityGroups",
        "ec2:DescribeSubnets",
        "ec2:DescribeTags",
        "ec2:DescribeVpcs",
        "ec2:ModifyInstanceAttribute",
        "ec2:ModifyNetworkInterfaceAttribute",
        "ec2:RevokeSecurityGroupIngress"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "A

ポリシーを作成します。

In [12]:
row = !aws iam create-policy \
--policy-name ALBIngressControllerIAMPolicy \
--policy-document file://temp/iam-policy.json

import json

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
policy = json.loads(''.join(row))
pprint.pprint(policy)

{'Policy': {'Arn': 'arn:aws:iam::565991835156:policy/ALBIngressControllerIAMPolicy',
            'AttachmentCount': 0,
            'CreateDate': '2020-01-30T11:06:50Z',
            'DefaultVersionId': 'v1',
            'IsAttachable': True,
            'Path': '/',
            'PermissionsBoundaryUsageCount': 0,
            'PolicyId': 'ANPAYHR56AYKFFF6DG3LH',
            'PolicyName': 'ALBIngressControllerIAMPolicy',
            'UpdateDate': '2020-01-30T11:06:50Z'}}


In [13]:
ALBPolicyArn = policy['Policy']['Arn']
print(ALBPolicyArn)

arn:aws:iam::565991835156:policy/ALBIngressControllerIAMPolicy


ワーカーノード用の IAM ロール名を取得します。次のコマンドを使用して、aws-auth configmap を出力します。

system:nodes グループが割り当てられている rolearn 値のロール名を取り出します。クラスターの各ノードグループに 1 つの値が必要です。

In [10]:
row = !kubectl -n kube-system describe configmap aws-auth

import pprint
import re

m = re.search(r'arn:aws:iam::([0-9]+):role/(.*)', row[12])

pprint.pprint(m.group(2))

workerRolearn = m.group(2)


res_info['workerRolearn']=workerRolearn

'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5'


次のコマンドを使用して、前に識別した各ワーカーノードの IAM ロールに、新しい ALBIngressControllerIAMPolicy IAM ポリシーをアタッチします。

In [16]:
!aws iam attach-role-policy \
--policy-arn {ALBPolicyArn} \
--role-name {workerRolearn}

VPC IDを記録しておく



In [21]:
vpcID = publicSubnets['Subnets'][0]['VpcId']



res_info['vpcID']=vpcID

●●●リソース情報の保存●●●

In [22]:
f = open('res_info.json', 'w')
json.dump(res_info, f)
f.close()